# 我們使用多層感知器完成 MNIST 手寫體數字圖片資料集 [LeCun1998] 的分類任務。
<img src="https://tf.wiki/_images/mnist_0-9.png">

# 資料獲取及預處理： tf.keras.datasets
## 先進行預備工作，實現一個簡單的 MNISTLoader 類來讀取 MNIST 資料集資料。這裡使用了 tf.keras.datasets 快速載入 MNIST 資料集。

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的圖片預設為uint8（0-255的數字）。以下程式碼將其正規化到0-1之間的浮點數，並在最後增加一維作為顏色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 從資料集中隨機取出batch_size個元素並返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

In [6]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten層將除第一維（batch_size）以外的維度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)   # 正規化output
        return output

<img src="https://tf.wiki/_images/mlp.png" height="50%">
MLP模型示意圖

## 模型的訓練： tf.keras.losses 和 tf.keras.optimizer
### 定義一些模型超參數 (Hyperparameters)：

In [4]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [7]:
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

11493376/11490434 [==============================] - 8s 1us/step


### 然後疊代進行以下步驟：

1.從 DataLoader 中隨機取一批訓練資料；

2.將這批資料送入模型，計算出模型的預測值；

3.將模型預測值與真實值進行比較，計算損失函數（loss）。這里使用 tf.keras.losses 中的交叉熵函數作為損失函數；

4.計算損失函數關於模型變數的導數；

5.將求出的導數值傳入優化器，使用優化器的 apply_gradients 方法更新模型參數以最小化損失函數

In [9]:
num_batches = int(data_loader.num_train_data // batch_size*num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.291271
batch 1: loss 2.285223
batch 2: loss 2.141097
batch 3: loss 2.163988
batch 4: loss 2.023757
batch 5: loss 1.885106
batch 6: loss 1.926385
batch 7: loss 1.878227
batch 8: loss 1.735710
batch 9: loss 1.720476
batch 10: loss 1.612328
batch 11: loss 1.546751
batch 12: loss 1.634239
batch 13: loss 1.496735
batch 14: loss 1.301871
batch 15: loss 1.401961
batch 16: loss 1.388678
batch 17: loss 1.301142
batch 18: loss 1.143222
batch 19: loss 1.224102
batch 20: loss 1.053555
batch 21: loss 1.130087
batch 22: loss 1.026671
batch 23: loss 1.099036
batch 24: loss 1.092223
batch 25: loss 1.117822
batch 26: loss 1.011977
batch 27: loss 0.954472
batch 28: loss 0.943555
batch 29: loss 0.938289
batch 30: loss 0.786768
batch 31: loss 0.912663
batch 32: loss 0.764814
batch 33: loss 1.003664
batch 34: loss 0.796574
batch 35: loss 0.696048
batch 36: loss 0.749329
batch 37: loss 0.895259
batch 38: loss 0.648174
batch 39: loss 0.679414
batch 40: loss 0.690644
batch 41: loss 0.552685
ba

batch 354: loss 0.357783
batch 355: loss 0.237268
batch 356: loss 0.222045
batch 357: loss 0.150385
batch 358: loss 0.190043
batch 359: loss 0.250200
batch 360: loss 0.267669
batch 361: loss 0.610833
batch 362: loss 0.176466
batch 363: loss 0.207088
batch 364: loss 0.464804
batch 365: loss 0.155659
batch 366: loss 0.196585
batch 367: loss 0.331554
batch 368: loss 0.226854
batch 369: loss 0.227246
batch 370: loss 0.226169
batch 371: loss 0.280046
batch 372: loss 0.112335
batch 373: loss 0.211127
batch 374: loss 0.273033
batch 375: loss 0.246294
batch 376: loss 0.297784
batch 377: loss 0.485503
batch 378: loss 0.457531
batch 379: loss 0.217041
batch 380: loss 0.312339
batch 381: loss 0.325612
batch 382: loss 0.392717
batch 383: loss 0.351575
batch 384: loss 0.312367
batch 385: loss 0.271365
batch 386: loss 0.300696
batch 387: loss 0.319081
batch 388: loss 0.330608
batch 389: loss 0.408813
batch 390: loss 0.257034
batch 391: loss 0.117344
batch 392: loss 0.315197
batch 393: loss 0.318378


batch 718: loss 0.129815
batch 719: loss 0.100344
batch 720: loss 0.152395
batch 721: loss 0.237098
batch 722: loss 0.236649
batch 723: loss 0.199645
batch 724: loss 0.254671
batch 725: loss 0.111365
batch 726: loss 0.328476
batch 727: loss 0.233017
batch 728: loss 0.216337
batch 729: loss 0.231799
batch 730: loss 0.343402
batch 731: loss 0.132605
batch 732: loss 0.267495
batch 733: loss 0.186177
batch 734: loss 0.314839
batch 735: loss 0.182717
batch 736: loss 0.178240
batch 737: loss 0.333411
batch 738: loss 0.215672
batch 739: loss 0.362835
batch 740: loss 0.277073
batch 741: loss 0.137396
batch 742: loss 0.225965
batch 743: loss 0.140093
batch 744: loss 0.158981
batch 745: loss 0.249106
batch 746: loss 0.260568
batch 747: loss 0.097931
batch 748: loss 0.251539
batch 749: loss 0.162105
batch 750: loss 0.335942
batch 751: loss 0.105484
batch 752: loss 0.221126
batch 753: loss 0.228851
batch 754: loss 0.148505
batch 755: loss 0.166845
batch 756: loss 0.282766
batch 757: loss 0.136153


batch 1061: loss 0.284547
batch 1062: loss 0.072142
batch 1063: loss 0.225128
batch 1064: loss 0.382878
batch 1065: loss 0.113176
batch 1066: loss 0.229559
batch 1067: loss 0.086311
batch 1068: loss 0.292671
batch 1069: loss 0.106893
batch 1070: loss 0.282904
batch 1071: loss 0.393813
batch 1072: loss 0.098925
batch 1073: loss 0.114219
batch 1074: loss 0.185054
batch 1075: loss 0.177528
batch 1076: loss 0.241266
batch 1077: loss 0.093640
batch 1078: loss 0.139963
batch 1079: loss 0.200847
batch 1080: loss 0.359671
batch 1081: loss 0.495648
batch 1082: loss 0.097195
batch 1083: loss 0.153848
batch 1084: loss 0.283950
batch 1085: loss 0.337289
batch 1086: loss 0.159291
batch 1087: loss 0.378476
batch 1088: loss 0.094318
batch 1089: loss 0.207743
batch 1090: loss 0.241167
batch 1091: loss 0.170425
batch 1092: loss 0.095858
batch 1093: loss 0.239252
batch 1094: loss 0.504507
batch 1095: loss 0.132698
batch 1096: loss 0.175934
batch 1097: loss 0.227630
batch 1098: loss 0.091253
batch 1099: 

batch 1414: loss 0.034338
batch 1415: loss 0.057500
batch 1416: loss 0.207829
batch 1417: loss 0.082114
batch 1418: loss 0.109324
batch 1419: loss 0.028773
batch 1420: loss 0.117327
batch 1421: loss 0.119261
batch 1422: loss 0.218370
batch 1423: loss 0.084706
batch 1424: loss 0.081975
batch 1425: loss 0.354730
batch 1426: loss 0.102844
batch 1427: loss 0.288735
batch 1428: loss 0.182819
batch 1429: loss 0.220233
batch 1430: loss 0.147184
batch 1431: loss 0.184934
batch 1432: loss 0.175797
batch 1433: loss 0.090687
batch 1434: loss 0.180390
batch 1435: loss 0.130301
batch 1436: loss 0.282021
batch 1437: loss 0.107972
batch 1438: loss 0.130068
batch 1439: loss 0.060448
batch 1440: loss 0.172176
batch 1441: loss 0.103325
batch 1442: loss 0.149747
batch 1443: loss 0.143962
batch 1444: loss 0.189824
batch 1445: loss 0.165536
batch 1446: loss 0.350154
batch 1447: loss 0.129056
batch 1448: loss 0.088090
batch 1449: loss 0.148904
batch 1450: loss 0.062710
batch 1451: loss 0.188165
batch 1452: 

...

batch 4664: loss 0.037863
batch 4665: loss 0.056191
batch 4666: loss 0.019976
batch 4667: loss 0.045029
batch 4668: loss 0.048523
batch 4669: loss 0.106222
batch 4670: loss 0.028294
batch 4671: loss 0.107870
batch 4672: loss 0.040794
batch 4673: loss 0.033968
batch 4674: loss 0.093684
batch 4675: loss 0.091062
batch 4676: loss 0.140612
batch 4677: loss 0.092995
batch 4678: loss 0.056692
batch 4679: loss 0.036963
batch 4680: loss 0.039960
batch 4681: loss 0.079320
batch 4682: loss 0.027891
batch 4683: loss 0.089336
batch 4684: loss 0.025866
batch 4685: loss 0.023713
batch 4686: loss 0.290515
batch 4687: loss 0.035081
batch 4688: loss 0.066395
batch 4689: loss 0.055181
batch 4690: loss 0.005580
batch 4691: loss 0.020488
batch 4692: loss 0.048678
batch 4693: loss 0.115098
batch 4694: loss 0.018545
batch 4695: loss 0.045275
batch 4696: loss 0.016563
batch 4697: loss 0.025135
batch 4698: loss 0.021856
batch 4699: loss 0.039251
batch 4700: loss 0.031008
batch 4701: loss 0.048674
batch 4702: 

batch 5014: loss 0.034808
batch 5015: loss 0.092139
batch 5016: loss 0.159522
batch 5017: loss 0.109472
batch 5018: loss 0.108439
batch 5019: loss 0.043243
batch 5020: loss 0.071433
batch 5021: loss 0.168188
batch 5022: loss 0.034611
batch 5023: loss 0.073439
batch 5024: loss 0.029745
batch 5025: loss 0.037333
batch 5026: loss 0.069103
batch 5027: loss 0.134612
batch 5028: loss 0.140184
batch 5029: loss 0.018722
batch 5030: loss 0.063659
batch 5031: loss 0.233953
batch 5032: loss 0.093950
batch 5033: loss 0.029139
batch 5034: loss 0.121729
batch 5035: loss 0.008188
batch 5036: loss 0.038277
batch 5037: loss 0.179696
batch 5038: loss 0.030490
batch 5039: loss 0.087076
batch 5040: loss 0.086278
batch 5041: loss 0.009447
batch 5042: loss 0.064277
batch 5043: loss 0.048913
batch 5044: loss 0.070227
batch 5045: loss 0.057417
batch 5046: loss 0.009790
batch 5047: loss 0.021420
batch 5048: loss 0.030615
batch 5049: loss 0.119441
batch 5050: loss 0.032021
batch 5051: loss 0.043045
batch 5052: 

batch 5332: loss 0.050604
batch 5333: loss 0.079479
batch 5334: loss 0.146943
batch 5335: loss 0.037309
batch 5336: loss 0.084309
batch 5337: loss 0.008781
batch 5338: loss 0.046345
batch 5339: loss 0.055581
batch 5340: loss 0.019918
batch 5341: loss 0.021501
batch 5342: loss 0.009232
batch 5343: loss 0.047443
batch 5344: loss 0.015879
batch 5345: loss 0.052334
batch 5346: loss 0.068811
batch 5347: loss 0.049911
batch 5348: loss 0.015884
batch 5349: loss 0.018970
batch 5350: loss 0.139109
batch 5351: loss 0.399518
batch 5352: loss 0.075439
batch 5353: loss 0.097404
batch 5354: loss 0.022765
batch 5355: loss 0.084896
batch 5356: loss 0.100513
batch 5357: loss 0.056473
batch 5358: loss 0.023080
batch 5359: loss 0.024391
batch 5360: loss 0.128763
batch 5361: loss 0.073707
batch 5362: loss 0.061900
batch 5363: loss 0.075444
batch 5364: loss 0.009120
batch 5365: loss 0.024845
batch 5366: loss 0.017929
batch 5367: loss 0.039923
batch 5368: loss 0.035021
batch 5369: loss 0.004967
batch 5370: 

batch 5650: loss 0.060755
batch 5651: loss 0.167001
batch 5652: loss 0.029364
batch 5653: loss 0.015593
batch 5654: loss 0.118595
batch 5655: loss 0.045563
batch 5656: loss 0.056672
batch 5657: loss 0.020753
batch 5658: loss 0.074766
batch 5659: loss 0.022439
batch 5660: loss 0.011773
batch 5661: loss 0.042044
batch 5662: loss 0.040171
batch 5663: loss 0.011396
batch 5664: loss 0.018046
batch 5665: loss 0.096333
batch 5666: loss 0.050647
batch 5667: loss 0.035649
batch 5668: loss 0.011951
batch 5669: loss 0.053682
batch 5670: loss 0.078821
batch 5671: loss 0.008807
batch 5672: loss 0.024259
batch 5673: loss 0.046202
batch 5674: loss 0.038896
batch 5675: loss 0.014147
batch 5676: loss 0.180039
batch 5677: loss 0.016855
batch 5678: loss 0.063730
batch 5679: loss 0.006294
batch 5680: loss 0.118167
batch 5681: loss 0.013878
batch 5682: loss 0.006805
batch 5683: loss 0.120820
batch 5684: loss 0.019146
batch 5685: loss 0.044405
batch 5686: loss 0.074491
batch 5687: loss 0.050414
batch 5688: 

batch 5966: loss 0.103853
batch 5967: loss 0.006550
batch 5968: loss 0.007081
batch 5969: loss 0.019017
batch 5970: loss 0.005650
batch 5971: loss 0.058320
batch 5972: loss 0.028373
batch 5973: loss 0.032608
batch 5974: loss 0.013398
batch 5975: loss 0.014454
batch 5976: loss 0.016990
batch 5977: loss 0.034029
batch 5978: loss 0.080058
batch 5979: loss 0.107675
batch 5980: loss 0.042319
batch 5981: loss 0.068726
batch 5982: loss 0.033984
batch 5983: loss 0.042827
batch 5984: loss 0.026201
batch 5985: loss 0.008219
batch 5986: loss 0.055822
batch 5987: loss 0.010761
batch 5988: loss 0.016432
batch 5989: loss 0.242611
batch 5990: loss 0.055622
batch 5991: loss 0.029512
batch 5992: loss 0.023599
batch 5993: loss 0.089246
batch 5994: loss 0.078832
batch 5995: loss 0.071922
batch 5996: loss 0.035222
batch 5997: loss 0.049888
batch 5998: loss 0.019435
batch 5999: loss 0.053633


In [11]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.974300
